In [1]:
import os
print(os.getcwd())
import sys
sys.path.append('/home/huan/Documents/sparse-coarse-operator-master')
from libs.pde import *
from libs.rotated_laplacian import RotatedLaplacian
from libs.models import BasicNet, MultiHeadedAttention
import torch

device = 'cuda'
print('device:', device)

/home/huan/Documents/sparse-coarse-operator-master/tests/laplacian/fixed-xi
device: cuda


# 3-Level Attention 

In [2]:
seed = 4
list_eps = [1/10]
theta_low = np.pi/3
theta_high = np.pi/2
npts = 5
list_theta = np.linspace(theta_low, theta_high, npts)

In [3]:
train_grid_size = 31
k2 = 10
k3 = 15
same_vecs = False
PDE = RotatedLaplacian(train_grid_size,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

In [4]:
# LEVEL 2
seed = 2
print('seed:', seed)
set_global_seed(seed)
h = 8
dhid = 256
epochs = 2000
adam_decay_rate = 0 # adam weight decay
lr = 1e-4 # learning rate
lr_decay_rate = 1 # learning rate decay
lr_decay_step = 1000 # learning rate decay step size
init = 'xavier uniform' # "<name> <distribution>" name={'xavier', 'kaiming'} distribution={'uniform', 'normal'}

ncopy = 1
single_model = False # KEEP IT FALSE!! if true, only model_prob will be learned
softmax_topk = True
dropout = 0.3
enforce_stencil_symmetry = 'x'

model_prob_l2 = MultiHeadedAttention(h ,dhid, ncopy=ncopy, init=init, dropout=dropout)
model_value_l2 = MultiHeadedAttention(h, dhid, ncopy=ncopy, init=init, dropout=dropout)


model_prob_l2,model_value_l2 = PDE.train(PDE.A2_train, 
                                         PDE.s2_train,
                                         PDE.eig_vec2_train,
                                         model_prob_l2,
                                         model_value_l2,
                                         epochs,
                                         adam_decay_rate,
                                         lr,
                                         lr_decay_rate,
                                         lr_decay_step,
                                         device,
                                         single_model=single_model,
                                         softmax_on = softmax_topk,
                                         enforce_stencil_symmetry = enforce_stencil_symmetry,
                                         verbose = True)




seed: 2
 epoch:  0  loss:  tensor(0.3015, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0138, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [ ]:
# test level-2 model
test_seed = 101
set_global_seed(test_seed)
print('test seed:', test_seed)
sizes = [63,127,255,511]
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
test_grid_size = 511
max_levels = 2
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
accel = 'gmres'
dropout_on = False
enforce_stencil_symmetry = 'x'

# for test_grid_size in sizes:
#     set_global_seed(test_seed)
#     print('size:', test_grid_size)
num_iter_standard, num_iter_learning, thetaList, xs, xl = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            single_model=single_model,
                                                            softmax_on = softmax_topk,
                                                            enforce_stencil_symmetry = enforce_stencil_symmetry,
                                                            dropout_on=dropout_on,
                                                            top_accel = accel,
                                                            verbose = True)

test seed: 101


In [10]:
print(num_iter_standard)
print(num_iter_learning)
print(thetaList)

[18, 19, 12, 13, 22, 26, 14, 27, 23, 13]
[18, 19, 13, 18, 22, 26, 15, 27, 23, 14]
[1.3175832403822403, 1.3459984009543915, 1.062106621316638, 1.137006080384654, 1.4060077397590085, 1.4838249274481101, 1.2079246879670198, 1.5150922655774495, 1.4249970337589501, 1.1466493550566506]


## Level 3

In [28]:
# LEVEL 3
l3_seed = 62
# l3_seed = l3_seed + 1
print('seed:', l3_seed)
set_global_seed(l3_seed)
h3 = 8
dhid3 = 256
epochs3 = 1000
adam_decay_rate3 = 0 # adam weight decay
lr3 = 1e-4 # learning rate
lr_decay_rate3 = 1 # learning rate decay
lr_decay_step3 = 500 # learning rate decay step size
init3 = 'xavier uniform' 

ncopy3 = 1
single_model3 = False
softmax_topk3 = True
dropout3 = 0.3

model_prob_l3 = MultiHeadedAttention(h3 ,dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)
model_value_l3 = MultiHeadedAttention(h3, dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)

model_prob_l3,model_value_l3 = PDE.train(PDE.A3_train, 
                                         PDE.s3_train,
                                         PDE.eig_vec3_train,
                                         model_prob_l3,
                                         model_value_l3,
                                         epochs3,
                                         adam_decay_rate3,
                                         lr3,
                                         lr_decay_rate3,
                                         lr_decay_step3,
                                         device,
                                         single_model= single_model3,
                                         softmax_on = softmax_topk3,
                                         verbose = True)

seed: 62
 epoch:  0  loss:  tensor(0.1176, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0022, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0014, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0015, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  999  loss:  tensor(0.0013, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)


In [ ]:
test_seed = seed + 2
set_global_seed(test_seed)
print('test seed:', test_seed)
num_test = 20
models = {}
models['level2'] = model_prob_l2, model_value_l2
models['level3'] = model_prob_l3, model_value_l3
test_grid_size = 1023
max_levels = 3
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)

num_iter_standard, num_iter_learning, _, _ = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            single_model=single_model,
                                                            softmax_on = softmax_topk,
                                                            verbose = True)

test seed: 65
standard stencil iter:    94.65   standard stencil time:   15.00200423002243
learned stencil iter:    89.45   learned stencil time:   16.40577573776245


In [39]:
print(num_iter_standard)
print(num_iter_learning)

[75, 68, 72, 70, 74, 69, 89, 93, 84, 76, 76, 85, 70, 82, 85, 92, 90, 72, 72, 89]
[63, 60, 64, 65, 61, 64, 72, 75, 71, 65, 68, 72, 64, 70, 72, 75, 73, 62, 65, 73]


# MLP

In [72]:
# define problem
list_eps = [0.01]
theta_low = np.pi/6
theta_high = np.pi/4
list_theta = np.linspace(theta_low, theta_high, 10)

In [74]:
# seed += 1
print('seed:',seed)
set_global_seed(seed)
epochs = 2000
adam_decay_rate = 0 # adam weight decay
lr = 1e-3 # learning rate
lr_decay_rate = 0.5 # learning rate decay
lr_decay_step = 500 # learning rate decay step size
model_prob_l2 = BasicNet([64])
model_value_l2 = BasicNet([64])
k2 = 10
k3 = 15
same_vecs = False
single_model = False # if true, only model_prob will be learned. 
softmax_on = True # if true, turn on the softmax in the function top_k during sparsfication
problem = RotatedLaplacian(31,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

model_prob_l2,model_value_l2 = problem.train(problem.A2_train, 
                                             problem.s2_train,
                                             problem.eig_vec2_train,
                                             model_prob_l2,
                                             model_value_l2,
                                             epochs,
                                             adam_decay_rate,
                                             lr,
                                             lr_decay_rate,
                                             lr_decay_step,
                                             device,
                                             single_model=single_model,
                                             softmax_on = softmax_on,
                                             verbose = True)

seed: 36
 epoch:  0  loss:  tensor(6.7295, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0111, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0104, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0101, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0098, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1250  loss:  tensor(0.0096, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1500  loss:  tensor(0.0095, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1750  loss:  tensor(0.0094, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1999  loss:  tensor(0.0093, device='cuda:0', grad_fn=<AddBackward0>)
